## `MyRequestTransformer` Class
Extends `PioRequestTransformer` class.  Must implement the `transform_request()` method.

### Convert `json` -> `dict`

In [43]:
from pio_model import PioRequestTransformer

class MyRequestTransformer(PioRequestTransformer): 

    def __init__(self, 
                 *args,
                 **kwargs):
        PioRequestTransformer.__init__(self, 
                                       args, 
                                       kwargs)

        
        
# Inputs
#   raw json (binary)
# Outputs
#   TensorFlow PredictRequest (protobuf) 
def transform_request(self,
                      request,
                      *args,
                      **kwargs):
    import tensorflow as tf
    import predict_pb2
    import json
    import numpy as np

    PioRequestTransformer.transform_request(self,
                                            request,
                                            args,
                                            kwargs)
    print(request)
    request_str = request.decode('utf-8')
    request_json = json.loads(request_str)
    request_np = np.asarray([request_json['x_observed']])
    request_tensor_proto = tf.contrib.util.make_tensor_proto(request_np,
                                                             dtype=tf.float32)
    request = predict_pb2.PredictRequest()
    request.inputs['x_observed'].CopyFrom(inputs_tensor_proto)
    return request  

## `MyResponseTransformer` Class
Extends `PioResponseTransformer` class.  Must implement the `transform_response()` method.

### Convert `dict` -> `json`

In [44]:
from pio_model import PioResponseTransformer

class MyResponseTransformer(PioResponseTransformer):
    def __init__(self, 
                 *args,
                 **kwargs):

        PioResponseTransformer.__init__(self, 
                                        args, 
                                        kwargs)

        
    def transform_response(self,
                           response,
                           *args,
                           **kwargs):

        import tensorflow as tf
        import json
        import numpy as np
        response_np_array = tf.contrib.util.make_ndarray(response.outputs['y_pred'])
        return json.dumps({"y_pred": response_np_array.tolist()[0]})

## Train My Model

## Create Model Initializer
Must implement the `initialize_model()` method

In [45]:
from pio_model import PioModelInitializer

class MyModelInitializer(PioModelInitializer): 

    def __init__(self, 
                 *args,
                 **kwargs):

        PioModelInitializer.__init__(self, 
                                     args, 
                                     kwargs)

        
    def initialize_model(self,                        
                         *args,
                         **kwargs):

        PioModelInitializer.initialize_model(self, 
                                             args, 
                                             kwargs)
        return None 

## `MyModel` Class
Extends `PioModel` class.  Must implement `predict()` method.

In [46]:
from pio_model import PioModel

class MyModel(PioModel):

    
    def __init__(self, 
                 request_transformer, 
                 response_transformer,
                 model_initializer,
                 *args,
                 **kwargs):

        PioModel.__init__(self,
                          request_transformer,
                          response_transformer,
                          model_initializer,
                          args,
                          kwargs)
        
    def predict(self, 
                request,
                *args,
                **kwargs):

        #transformed_request = self.request_transformer.transform_request(request)
        
        #response = self.model.predict(transformed_request)
        
        #transformed_response = self.response_transformer.transform_response(response)
        
        #return transformed_response
        return 

## Construct `MyModel`
Inject Model with Request Transformer, Response Transformer, and Model Initializer
```
MyModel(`MyRequestTransformer`, `MyResponseTransformer`, `MyModelInitializer`)
```

In [47]:
pio_model = MyModel(MyRequestTransformer(),
                    MyResponseTransformer(), 
                    MyModelInitializer())

## Save/Pickle Pio Model as `pio_model.pkl`

In [50]:
import dill as pickle

pio_model_pkl_path = 'pio_model.pkl'

with open(pio_model_pkl_path, 'wb') as fh:
    pickle.dump(pio_model, fh)
        
with open(pio_model_pkl_path, 'rb') as fh:
    pio_model = pickle.load(fh)

import json

json_request = json.loads('{"x_observed":1.5}')
print(json_request)    

{'x_observed': 1.5}


In [51]:
import subprocess

output = subprocess.check_output('ls -l pio_model.pkl',
                                 stderr=subprocess.STDOUT,
                                 shell=True)
print(output.decode('utf-8'))

-rw-r--r-- 1 root root 2142 Jun 15 21:48 pio_model.pkl

